In [1]:
!pip install pandas
!pip install request

import pandas as pd
import unicodedata
import datetime

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Exception:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\commands\install.py", line 342, in run
    prefix=options.prefix_path,
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\req\req_set.py", line 784, in install
    **kwargs
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\req\req_install.py", line 851, in install
    self.move_wheel_files(self.source_dir, root=root, prefix=prefix)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\req\req_install.py", line 1064, in move_wheel_files
    isolated=self.isolated,
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\wheel.py", line 347, in move_wheel_files
    assert info_dir, "%s .dist-info directory not found" % req
AssertionError: request .dist-info directory not found
You are using pip version 9.0.1, however version 10.0.1 is available.
You should

In [2]:
url_componentes = 'componentes-graduacao.csv'
url_unidades = 'http://dados.ufrn.br/dataset/da6451a5-1a59-4630-bdc2-97f6be4a59c2/resource/3f2e4e32-ef1a-4396-8037-cbc22a89d97f/download/unidades.csv'
url_docentes = 'http://dados.ufrn.br/dataset/8bf1a468-48ff-4f4d-95ee-b17b7a3a5592/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070/download/docentes.csv'
url_avaliacoes = 'http://dados.ufrn.br/dataset/d5723d75-7e6e-4264-82aa-b96909b69f63/resource/7accd1d2-2793-460e-b98d-87a0679b9155/download/avaliacaodocencia.csv'

In [3]:
df_unidades = pd.read_csv(url_unidades, ';')
df_unidades = df_unidades[['id_unidade_responsavel', 'unidade_responsavel']]
df_unidades.rename(columns={'id_unidade_responsavel': 'id_unidade', 'unidade_responsavel': 'lotacao'}, inplace=True)
df_unidades = df_unidades[(df_unidades['lotacao'] == 'DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA') | (df_unidades['lotacao'] == 'INSTITUTO METROPOLE DIGITAL')].drop_duplicates()
df_unidades.reset_index(inplace=True, drop=True)
df_unidades.head()

,id_unidade,lotacao
0,125,DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA
1,6069,INSTITUTO METROPOLE DIGITAL


In [4]:
df_docentes = pd.read_csv(url_docentes, ';')
df_docentes.columns

Index([u'id_servidor', u'siape', u'nome', u'formacao',
       u'tipo_jornada_trabalho', u'vinculo', u'categoria', u'classe_funcional',
       u'id_unidade_lotacao', u'lotacao', u'admissao', u'Unnamed: 11'],
      dtype='object')

In [5]:
df_docentes = df_docentes[df_docentes['categoria'] == 'PROFESSOR DO MAGISTERIO SUPERIOR']

In [6]:
df_docentes = df_docentes[['id_servidor', 'nome', 'formacao', 'id_unidade_lotacao', 'admissao']]
df_docentes.rename(columns={'id_servidor': 'id_docente', 'admissao': 'data_admissao', 'id_unidade_lotacao': 'id_unidade'}, inplace=True)
df_docentes = df_docentes.merge(df_unidades[['id_unidade']], on='id_unidade', how='inner')
df_docentes['data_admissao'] = [ data.date() for data in pd.to_datetime(df_docentes['data_admissao'])]
df_docentes['id_unidade'].value_counts()

125     35
6069    27
Name: id_unidade, dtype: int64

In [7]:
df_componentes = pd.read_csv(url_componentes, sep=',')
df_componentes = df_componentes[(df_componentes['unidade_responsavel'] == 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA') | (df_componentes['unidade_responsavel'] == 'INSTITUTO METROPOLE DIGITAL')]
df_componentes.head()

,id_componente,codigo,nivel,nome,unidade_responsavel
6,20007,DIM0072,G,TECNICAS DE ESPECIFICACAO FORMAL,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1033,21073,DIM0056,G,SOFTWARE BASICO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1043,21083,DIM0346,G,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1619,21667,DIM0046,G,ALGORITMOS E TECNICAS DE PROGRAMACAO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1620,21668,DIM0003,G,ORGANIZACAO DE COMPUTADORES,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA


In [8]:
unidades_normalizadas = [unicodedata.normalize('NFKD', unicode(nome, "utf-8")).encode('ASCII', 'ignore') for nome in df_componentes['unidade_responsavel']]
series = pd.Series(data=unidades_normalizadas, index=df_componentes.index)
df_componentes['unidade_responsavel'] = series

In [9]:
df_componentes = df_componentes.merge(df_unidades[['id_unidade', 'lotacao']], left_on='unidade_responsavel', right_on='lotacao', how='inner')

In [10]:
df_componente_curricular = df_componentes[['id_componente', 'codigo']]
df_componente_curricular.head()

,id_componente,codigo
0,20007,DIM0072
1,21073,DIM0056
2,21083,DIM0346
3,21667,DIM0046
4,21668,DIM0003


In [11]:
df_componente_lotacao = df_componentes[['codigo', 'id_unidade', 'nome']]
df_componente_lotacao.head()

,codigo,id_unidade,nome
0,DIM0072,125,TECNICAS DE ESPECIFICACAO FORMAL
1,DIM0056,125,SOFTWARE BASICO
2,DIM0346,125,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...
3,DIM0046,125,ALGORITMOS E TECNICAS DE PROGRAMACAO
4,DIM0003,125,ORGANIZACAO DE COMPUTADORES


In [12]:
turma_periodo = ['http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e6e4144f-4042-4fdc-84e0-76e9ec27ae7c/download/turmas-2014.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/2c69547b-920f-4ec2-92c0-3fbc19512165/download/turmas-2014.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/0d96d930-8058-4def-9044-c3ae04c1f40c/download/turmas-2013.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e7e3cf12-a29b-491a-a895-021a43819197/download/turmas-2013.2.csv',]

In [13]:
frames = [pd.read_csv(periodo, sep=';', usecols=['id_turma', 'ano', 'nivel_ensino', 'periodo', 'id_componente_curricular']) for periodo in turma_periodo]

In [14]:
df_turmas = pd.concat(frames)

In [15]:
df_turmas.rename(columns={'nivel_ensino': 'nivel', 'id_componente_curricular': 'id_componente'}, inplace=True)
df_turmas = df_turmas[df_turmas['nivel'] == 'GRADUAÇÃO']
df_turmas['periodo'] = df_turmas['periodo'].astype(int) 
df_turmas['id_componente'] = df_turmas['id_componente'].astype(int)
df_turmas = df_turmas.merge(df_componente_curricular[['id_componente']], on='id_componente', how='inner')
df_turmas.head()

,id_turma,id_componente,nivel,ano,periodo
0,57596647,2022854,GRADUAÇÃO,2017,1
1,57606915,2022854,GRADUAÇÃO,2017,2
2,57586681,2022854,GRADUAÇÃO,2016,2
3,1223534,2022854,GRADUAÇÃO,2014,1
4,1233980,2022854,GRADUAÇÃO,2014,2


In [16]:
df_avaliacoes = pd.read_csv(url_avaliacoes, sep=';', usecols=['id_docente', 'id_turma', 'qtd_discentes', 'postura_profissional_media', 'postura_profissional_DP', 'atuacao_profissional_media', 'atuacao_profissional_DP'])
df_avaliacoes = df_avaliacoes.merge(df_turmas[['id_turma']].astype(int), on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_docentes[['id_docente']].astype(int), on='id_docente', how='inner')
df_avaliacoes.rename(columns={'postura_profissional_media': 'postura_profissional', 'atuacao_profissional_media': 'atuacao_profissional'}, inplace=True)
df_avaliacoes.head()

,id_docente,id_turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP
0,22764,1233659,17,9.09,1.57,9.00,1.60
1,22764,1233661,36,9.48,1.26,9.43,1.20
2,22764,1242644,34,9.41,1.04,8.86,1.73
3,22764,57566712,36,9.81,0.51,9.48,1.35
4,22764,57566818,14,9.69,0.48,8.13,2.44


In [17]:
matriculas = [
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/55dfe713-ff7c-4fa8-8d1d-d4294a025bff/download/matricula-componente-20172',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/f6179838-b619-4d7d-af9c-18c438b80dd4/download/matriculas-de-2016.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/4778d3ce-8898-46a8-a623-ee6a480a2980/download/matriculas-de-2016.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/baa6c8b4-2072-417f-b238-c028ccc8c14b/download/matriculas-de-2015.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/9e7ba1c2-f92d-4b9c-9e91-3b026ecdf913/download/matriculas-de-2015.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/e974792c-b557-470c-bf3d-ede7d5b5e6a6/download/matricula-componente-20142.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/7081446d-39f9-4374-ad0b-86ecab97e569/download/matricula-componente-20141.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/517ed5f6-f8a2-40fd-826b-6ed3388f6e88/download/matricula-componente-20132.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/674de4cc-1fc0-4314-9f04-a38f0e1f9225/download/matricula-componente-20131.csv',
]

In [18]:
dfs_matricula = [pd.read_csv(matricula, sep=';') for matricula in matriculas]

In [19]:
df_matriculas = pd.concat(dfs_matricula)

C:\Users\paulohq\AppData\Local\conda\conda\envs\ipykernel_py2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [20]:
df_matriculas = df_matriculas[~df_matriculas['descricao'].isin(['CUMPRIU', 'INDEFERIDO', 'EXCLUIDA'])]

In [21]:
df_matriculas.replace('APROVADO POR NOTA', 'APROVADO', inplace=True)
df_matriculas.replace(['REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 'REPROVADO POR MÉDIA E POR FALTAS', 'REPROVADO POR NOTA E FALTA', 'DESISTENCIA', 'TRANCADO', 'CANCELADO'], 'REPROVADO', inplace=True)

In [22]:
df_matriculas = df_matriculas.merge(df_turmas[['id_turma']].astype(int), on='id_turma', how='inner')
df_matriculas.head()

,Unnamed: 10,descricao,discente,faltas_unidade,id_curso,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade
0,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.8,0.0,f,2.0
1,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.3,0.0,f,1.0
2,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.0,0.0,f,3.0
3,NaN,APROVADO,853a80ecb864882511c390f74aebbdfb,0.0,2000069.0,57606947,7.0,6.2,8.0,f,3.0
4,NaN,APROVADO,853a80ecb864882511c390f74aebbdfb,0.0,2000069.0,57606947,7.0,7.5,8.0,f,2.0


In [23]:
df_aprovados = df_matriculas[df_matriculas['descricao'] == 'APROVADO']
df_media_aprovados = df_aprovados.groupby('id_turma')['media_final'].mean().to_frame('media_aprovados').reset_index(level=0)

df_media_aprovados.head()

,id_turma,media_aprovados
0,1197615,7.010204
1,1197616,7.031250
2,1198464,6.727273
3,1198466,7.208696
4,1198499,6.451852


In [24]:
#media_turma = df_matriculas.groupby('id_turma')['media_final'].mean().to_frame('média').reset_index(level=0)
#media_turma.head()

In [25]:
df_porcentagem = df_matriculas.groupby('id_turma')['descricao'].value_counts(normalize=True).to_frame('aprovados')

In [26]:
df_porcentagem = df_porcentagem.reset_index(level=[0, 1])
df_porcentagem = df_porcentagem[~df_porcentagem['descricao'].isin(['REPROVADO'])]
df_porcentagem = df_porcentagem[['id_turma', 'aprovados']]
df_porcentagem.head()

,id_turma,aprovados
0,1197615,0.840000
2,1197616,0.578313
4,1198464,0.846154
7,1198466,0.482517
9,1198499,0.462857


In [27]:
df_avaliacoes = df_avaliacoes.merge(df_media_aprovados, on='id_turma', how='left')
df_avaliacoes = df_avaliacoes.merge(df_porcentagem, on='id_turma', how='left')
df_avaliacoes.head()

,id_docente,id_turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP,media_aprovados,aprovados
0,22764,1233659,17,9.09,1.57,9.00,1.60,6.670588,0.739130
1,22764,1233661,36,9.48,1.26,9.43,1.20,6.842857,0.882353
2,22764,1242644,34,9.41,1.04,8.86,1.73,7.603030,0.916667
3,22764,57566712,36,9.81,0.51,9.48,1.35,7.578788,0.860870
4,22764,57566818,14,9.69,0.48,8.13,2.44,7.458333,0.837209


In [28]:
df_avaliacoes.to_csv('avaliacao.csv', sep=';', index=False)
df_componente_lotacao.to_csv('componente_lotacao.csv', sep=';', index=False)
df_componente_curricular.to_csv('componente_curricular.csv', sep=';', index=False)
df_docentes.to_csv('docente.csv', sep=';', index=False)
df_unidades.to_csv('unidade.csv', sep=';', index=False)
df_turmas.to_csv('turma.csv', sep=';', index=False)

In [1]:
import pandas as pd
import requests
import json

api_url = 'http://localhost:8080/'

def make_post(df, endpoint):
    records = json.loads(df.to_json(orient='records'))
    for record in records:
        request = requests.post(endpoint, data=json.dumps(record), headers={'Content-Type': 'application/json'})
        #print request.text
        if not request.ok:
            break

def insert(df_url, endpoint):
    df = pd.read_csv(df_url, sep=';')
    make_post(df, endpoint)

def insert_componentes():
    df_lotacao = pd.read_csv('componente_lotacao.csv', sep=';')
    df_curricular = pd.read_csv('componente_curricular.csv', sep=';')
    df = df_lotacao.merge(df_curricular, on='codigo')
    make_post(df, api_url+'componente')

if __name__ == '__main__':
    #insert('unidade.csv', api_url+'unidade')
    #insert('docente.csv', api_url+'docente')
    #insert_componentes()
    #insert('turma.csv', api_url+'turma')
    insert('avaliacao.csv', api_url+'avaliacao')